In [1]:
import tensorwrap as tf
model = tf.nn.Sequential([
    tf.nn.layers.Dense(10),
    tf.nn.layers.Dense(1)
])
model.compile(
    loss=tf.nn.losses.mse,
    optimizer=tf.nn.optimizers.gradient_descent(),
    metrics=tf.nn.losses.mae
)

In [2]:
model.layers[0].trainable_variables

[]

In [3]:
x = tf.range(1, int(1e5))
x = tf.expand_dims(x, axis=1)
model.fit(x, x + 10, epochs=1, verbose=2)

Epoch 1|1 			 Loss: 3038639872.00000			     Metric: 47729.98438


In [4]:
model.trainable_variables

[[],
 [Array([[955.27966, 954.99445, 955.4784 , 954.91437, 954.671  , 955.25824,
          955.0494 , 955.0358 , 955.4092 , 955.00757]], dtype=float32),
  Array([954.5997, 954.5997, 954.5997, 954.5997, 954.5997, 954.5997,
         954.5997, 954.5997, 954.5997, 954.5997], dtype=float32)]]